<a href="https://colab.research.google.com/github/jan-kreischer/UZH_ML4NLP/blob/main/Project-06/ex06_tm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Project 6 - Topic Modeling using Latent Dirichlet Allocation (LDA)
## 1. Setup
### 1.1 Dependencies
Installing all dependencies needed to run the simulations

In [ ]:
!pip install contextualized-topic-models==2.2.0

### 1.2 Imports

In [27]:
import re
import random
import os
import urllib
import urllib.request
import gzip
import io
import csv
import random
from collections import defaultdict
from tqdm import tqdm

### 1.3 Google Drive
We connect Google Drive in order to access stored data.

In [3]:
# Enable access to files stored in Google Drive
from google.colab import drive
# Leave this like it is
mountpoint = '/content/drive/' 
drive.mount(mountpoint)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# Adapt this path to the folder where your data is stored in google drive
base_path = 'My Drive/UZH_ML4NLP/Projects/Project-06/data' 
data_path = os.path.join(mountpoint, base_path)
# Cd into the directory with the git repo
% cd $data_path

/content/drive/My Drive/UZH_ML4NLP/Projects/Project-06/data


### 1.4 Constants


In [41]:
NUM_LDA_TOPICS = 5 # The number of different topics to identify
NUM_FEATURES = 10000
MAX_DF=0.5
MIN_DF=0.01

In [5]:
# Path to the data files
path_before_1990 = 'titles_before_1990.txt'
path_from_1990_to_2009 = 'titles_from_1990_to_2009.txt'
path_from_2010 = 'titles_from_2010.txt'

In [6]:
# Execute the following cell only once to download the data and write it as a file to your google drive. Afterwards, skip this cell or comment it out.
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# to download the data manually or get more information, go to: https://dblp.org/faq/How+can+I+download+the+whole+dblp+dataset.html
url = 'https://dblp.uni-trier.de/xml/dblp.xml.gz'
num_titles = 500000  # the (max)number of titles to load 


def load_gzip_file(url):
    """Download Gzip-file."""
    response = urllib.request.urlopen(url)
    compressed_file = io.BytesIO(response.read())
    decompressed_file = gzip.GzipFile(fileobj=compressed_file)
    return decompressed_file

def extract_titles(input_file, max_num=40000):
    """Extract title and publication year of dblp papers, given as input file.
    
    Divide the papers into 3 time periods. 
    
    Collect max max_num papers per time period.
    """
    pairs_before_1990 = []
    count_before_1990 = 0
    pairs_from_1990_to_2009 = []
    count_from_1990_to_2009 = 0
    pairs_from_2010 = []
    count_from_2010 = 0
    got_title = False
    for line in tqdm(input_file):
        line_str = line.decode('utf-8')
        if got_title: 
            # we have a title and check for the corresponding year
            year_result = re.search(r'<year>(.*)</year>', line_str)
            if year_result:
                # we also have the year and thus save the title-year pair
                year = int(year_result.group(1))
                if year < 1990:
                    pairs_before_1990.append((title, year))
                    count_before_1990 += 1
                elif year < 2010:
                    pairs_from_1990_to_2009.append((title, year))
                    count_from_1990_to_2009 += 1
                else:
                    pairs_from_2010.append((title, year))
                    count_from_2010 += 1
                got_title = False
        else:
            # we have no title and search for title
            result = re.search(r'<title>(.*)</title>', line_str)
            if result:
                title = result.group(1)
                if len(title.split(' ')) < 3:  
                    # only include titles with at least four words
                    continue
                got_title = True
        
        if count_before_1990 >= max_num and count_from_1990_to_2009 >= max_num and count_from_2010 >= max_num:
            return pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010
    
    return pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010

def save_data(pairs, file_path):
    with open(file_path, 'w') as fout:
        writer = csv.writer(fout)
        for pair in pairs:
            writer.writerow(pair)

in_file = load_gzip_file(url)
pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010 = extract_titles(in_file)
save_data(pairs_before_1990, path_before_1990)
save_data(pairs_from_1990_to_2009, path_from_1990_to_2009)
save_data(pairs_from_2010, path_from_2010)
'''

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\', force_remount=True)\n\n# to download the data manually or get more information, go to: https://dblp.org/faq/How+can+I+download+the+whole+dblp+dataset.html\nurl = \'https://dblp.uni-trier.de/xml/dblp.xml.gz\'\nnum_titles = 500000  # the (max)number of titles to load \n\n\ndef load_gzip_file(url):\n    """Download Gzip-file."""\n    response = urllib.request.urlopen(url)\n    compressed_file = io.BytesIO(response.read())\n    decompressed_file = gzip.GzipFile(fileobj=compressed_file)\n    return decompressed_file\n\ndef extract_titles(input_file, max_num=40000):\n    """Extract title and publication year of dblp papers, given as input file.\n    \n    Divide the papers into 3 time periods. \n    \n    Collect max max_num papers per time period.\n    """\n    pairs_before_1990 = []\n    count_before_1990 = 0\n    pairs_from_1990_to_2009 = []\n    count_from_1990_to_2009 = 0\n    pairs_from_2010 = []\n    count_from_2010 = 

# LDA

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

num_lda_topics = 8

In [22]:
def load_titles(path):
  with open(path) as fin:
    reader = csv.reader(fin)
    titles = [row[0] for row in reader]
  return titles

### Before the 1990s:

In [39]:
titles_before_1990 = load_titles(path_before_1990)
print("{} titles before 1990".format(len(titles_before_1990)))
titles_from_1990_to_2009 = load_titles(path_from_1990_to_2009)
print("{} titles from 1990 to 2009".format(len(titles_from_1990_to_2009)))
titles_from_2010 = load_titles(path_from_2010)
print("{} titles from from 2010".format(len(titles_from_2010)))

40000 titles before 1990
327307 titles from 1990 to 2009
715820 titles from from 2010


In [29]:
random.sample(titles_before_1990, 10)

["Equality and Lyndon's Interpolation Theorem.",
 'FSPICE: a tool for fault modelling in MOS circuits.',
 'Minimal Degrees and the Jump Operator.',
 'Key note address.',
 "MMIC's in communications.",
 'A Probabilistic Algorithm for Vertex Connectivity of Graphs.',
 'Automorphism Groups of Primitive Distance-Bitransitive Graphs are Almost Simple.',
 'On Definitions of a Burst.',
 'Residual Life Approximations in General Queueing Networks.',
 'Fractal Nature of Software-Cache Interaction.']

In [30]:
random.sample(titles_from_1990_to_2009, 10)

['Global Optimization: Interval Analysis and Balanced Interval Arithmetic.',
 'Simulation of two-plane automatic balancing of a rigid rotor.',
 'On Enforced Convergence of ACO and its Implementation on the Reconfigurable Mesh Architecture Using Size Reduction Tasks.',
 'Approximating the Stable Model Semantics is Hard.',
 'Multiresolution representations for surfaces meshes based on the vertex decimation method.',
 'Motion analysis and estimation: From ill-posed discrete inverse linear problems to MPEG-2 coding.',
 '2-D Parachute Simulation by the Immersed Boundary Method.',
 'Wireless network directions.',
 'Reliability and Risk Models: Setting Reliability Requirements.',
 'A Paradigm for Listing (s, t)-Cuts in Graphs.']

In [31]:
random.sample(titles_from_2010, 10)

['Vacuum Constraints for Realistic Strongly Coupled Heterotic M-Theories.',
 'Error Correction Encoding for Tightly Coupled On-Chip Buses.',
 'Cyclic codes over R k.',
 '3-D Convolution-Recurrent Networks for Spectral-Spatial Classification of Hyperspectral Images.',
 'An arts-informed study of information using the draw-and-write technique.',
 'Assessments of the Retrieval of Atmospheric Profiles from GNSS Radio Occultation Data in Moist Tropospheric Conditions Using Radiosonde Data.',
 'Data-stories about (im)patient customers in tele-queues.',
 'Multi-Strategy Sentiment Analysis of Consumer Reviews Based on Semantic Fuzziness.',
 'A Strategy to Mitigate the Ionospheric Scintillation Effects on BDS Precise Point Positioning: Cycle-Slip Threshold Model.',
 'Ontology-Based Multiple Choice Question Generation.']

In [32]:
# Simple text preprocessing by removing 
# all letters which are not in roman alphabet
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z ]', '', text)
    text = text.lower()
    return text

In [34]:
preprocessed_titles_before_1990 = [preprocess_text(title) for title in titles]
preprocessed_titles_from_1990_to_2009 = [preprocess_text(title) for title in titles]
preprocessed_titles_from_2010 = [preprocess_text(title) for title in titles]

In [75]:
# Now we turn the documents (or titles in this case) into a matrix feature representation.
def vectorize_data(titles, max_df=MAX_DF, min_df=MIN_DF, max_features=NUM_FEATURES):
  tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df, max_features=max_features, stop_words='english')
  tf = tf_vectorizer.fit_transform(titles)
  tf_feature_names = tf_vectorizer.get_feature_names_out()
  return tf, tf_feature_names

In [79]:
tf, tf_feature_names = vectorize_data(preprocessed_titles_before_1990, max_df=0.95, min_df=0.01)

In [80]:
lda = LatentDirichletAllocation(n_components=8, max_iter=5, learning_method='online', random_state=42).fit(tf)

In [81]:
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {topic_idx}:', end=' ')
    print(' '.join([tf_feature_names[i] for i in topic.argsort()[:-12 - 1:-1]]))

Topic 0: theory problems algorithms simulation decision parallel application solution applications optimal digital control
Topic 1: computer logic model programs digital performance design networks using simulation applications systems
Topic 2: problem programming optimal digital language processing software research parallel solution linear control
Topic 3: data method network models application languages solution processing problem using analysis parallel
Topic 4: note linear information functions applications finite technical systems programming time sets problem
Topic 5: design analysis algorithm approach sets performance new using digital parallel implementation optimal
Topic 6: systems using parallel performance implementation decision distributed linear digital control design processing
Topic 7: control networks new recognition distributed time pattern optimal systems digital approach linear


In [21]:
#for i in range(num_lda_topics):
#    print(f'Topic {i + 1}:\t{lda.show_topic(i)}')

AttributeError: ignored

Topics:
0. Graph/networks algorithms (seems to be mostly about algorithms that (maybe) operate on graphs/networks)
1. pattern recognition (and maybe robotics)
2. ...

### From 1990 to 2009:

Add your code for topic modelling the period from 1990 to 2009 here...

In [36]:
num_features = 10000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=N_FEATURES, stop_words='english')
tf = tf_vectorizer.fit_transform(prepro_titles)
tf_feature_names = tf_vectorizer.get_feature_names_out()

### From 2010 onwards:

Add your code for topic modelling the period from 2010 onwards here...

# Combined Topic Models

New method developed by [Bianchi et al. 2021](https://aclanthology.org/2021.acl-short.96/). 

[A 6min presentation of the paper by one of the authors.](https://underline.io/lecture/25716-pre-training-is-a-hot-topic-contextualized-document-embeddings-improve-topic-coherence)

Code: [https://github.com/MilaNLProc/contextualized-topic-models](https://github.com/MilaNLProc/contextualized-topic-models)

Tutorial: [https://colab.research.google.com/drive/1fXJjr_rwqvpp1IdNQ4dxqN4Dp88cxO97?usp=sharing](https://colab.research.google.com/drive/1fXJjr_rwqvpp1IdNQ4dxqN4Dp88cxO97?usp=sharing)

Again, perform topic modelling for the three time periods - this time using the combined topic models (CTMs). 

You can use and adapt the code from the tutorial linked above.

Use the available GPU for faster running times.

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing

num_ctm_topics = 5  # you can also choose a higher number of topics

### Before the 1990s:

### From 1990 to 2009

### From 2010 onwards